In [1]:
import os
import pickle

import pandas as pd

In [2]:
seed = 28082017

In [ ]:
prediction_file = os.path.join('../intermediate/prediction_seed{}.pkl'
                               .format(seed))

with open(prediction_file, 'rb') as f:
    prediction = pickle.load(f)

In [ ]:
data_df = pd.read_csv('../data/train_1.csv', index_col='Page')
key_df = pd.read_csv('../data/key_1.csv')
key_df['Date'] = key_df['Page'].apply(lambda a: a[-10:])
key_df['Page'] = key_df['Page'].apply(lambda a: a[:-11])

In [ ]:
future_start = (pd.Timestamp('2017-01-01') - pd.Timestamp('2017-01-01')).days
future_end = (pd.Timestamp('2017-03-01') - pd.Timestamp('2017-01-01')).days
future_period = future_end + 1 - future_start

visits = np.zeros(key_df.shape[0])
for i in range(0, len(visits), future_period):
    page = key_df['Page'][i]
    page_index = data_df.index.get_loc(page)
    visits[i:(i+future_period)] = prediction[page_index,
                                             future_start:(future_end+1)]

key_df['Visits'] = visits
submission_file = os.path.join('../intermediate/submission_{}.csv'
                               .format(seed))
key_df[['Id', 'Visits']].to_csv(submission_file, index=False)